# Projections in PorePy (first draft, large changes might happen)

In this tutorial, the focus is to illustrate the use of the projection methods that
exist in PorePy. The projection of quantities is done between a pair of subdomains that
are exactly one dimension apart, such as e.g. a rock and a fracture. The transfer is
done via the use of specific projections to and from the interface between them, where
the type of projection method used depends on the type of quantities involved (extensive
or intensive properties). 

In PorePy, the following projection methods exist:

1. Projections used for Extensive Quantities:
-  `primary_to_mortar_int`
- `secondary_to_mortar_int`
- `mortar_to_primary_int`
- `mortar_to_secondary_int`

2. Projections used for Intensive Quantities:
- `mortar_to_primary_avg`
- `mortar_to_secondary_avg`
- `primary_to_mortar_avg`
- `secondary_to_mortar_avg`

## Terminology and concept
Assume that we have a fractured domain, like that shown in the left diagram below, which
is represented by a mixed-dimensional geometry. Quantities are projected between the
fracture (lower dimensional subdomain, colored in black) and the rock (higher
dimensional subdomain, colored in light grey) through an interface (right diagram,
dashed blue).

There is one grid/subdomain on each side of the interface, and in addition to being
referred to as subdomains, they are also termed primary and secondary grid. Typically,
the primary grid is of one dimension higher than the interface, while the secondary grid
is of the same dimension as the interface.

<img src='img/issue1235_frac_intf.png'  width=900>

We now want to project an arbitrary quantity $\gamma$ from the higher dimensional
subdomain (primary grid, $\Omega_h$) to the lower dimensional subdomain (secondary grid,
$\Omega_l$). The projection is done via the interface $\Gamma_j$, and it therefore
requires projecting in two-steps: first from the primary grid to the interface, and then
from the interface to the secondary grid. 

Mathematically, the process is described as follows: $$ \Omega_h\overset{\Pi_{j}^h
\gamma}{\longrightarrow} \Gamma_j\overset{{\Xi}_{l}^j (\beta)}{\longrightarrow}
\Omega_l, $$ where $$\beta = \Pi_{h}^j(\gamma).$$ Here:
- $\Pi_{j}^h$: is the projection map from $\Omega_h$ to $\Gamma_j,$
- $\Xi_{l}^j$: is the projection map from $\Gamma_j$ to $\Omega_l.$

For a more detailed explanation of the mathematics underlying these projections, please
refer to [this webpage](https://github.com/pmgbergen/porepy/issues/1235).


## Example of projection of an intensive quantity
We are now going to explore projecting an intensive quantity (e.g., pressure) from the
primary to the secondary grid. In practice that means that we will use the following two
projection methods: `primary_to_mortar_avg` and `mortar_to_secondary_avg`.

First we define the mixed-dimensional geometry which represent the fractured domain:


In [ ]:
import numpy as np
import porepy as pp

# Define a rectangular domain in terms of range in the two dimensions
bounding_box = {'xmin': 0, 'xmax': 3, 'ymin': 0, 'ymax': 3}
domain = pp.Domain(bounding_box=bounding_box)

# Define the fracture
fracture = pp.LineFracture(
    np.array(
        [
            [1.5, 1.5],
            [0.5, 2.5]
        ]
    )
)
# Define a fracture network in 2d
network_2d = pp.create_fracture_network([fracture], domain)

# Set overall target cell size and target cell size close to the fracture.
mesh_args: dict[str, float] = {"cell_size": 0.5, "cell_size_fracture": 0.1} 

# Generate a mixed-dimensional grid
mdg = pp.create_mdg("cartesian", mesh_args, network_2d)

We want to project "pressure" values, and we therefore generate an array of random
"pressure"-values:

In [32]:
for g in mdg.subdomains():
    if g.dim == 2:
        prim_faces = g.num_faces
primary_pressure = np.random.rand(prim_faces)

Now that the values are defined, we project them from the primary grid to the mortar
grid, and then from the mortar grid and to the secondary grid:

In [ ]:
# Fetch the interface/mortar grid
interface = mdg.interfaces()[0]

# Project from the primary grid to the mortar grid
mortar_pressure = interface.primary_to_mortar_avg() @ primary_pressure
# print("Pressure on the mortar grids:", mortar_pressure)

# Project from the mortar grid to the secondary grid
secondary_pressure = interface.mortar_to_secondary_avg() @ mortar_pressure
# print("Projected pressure on secondary grid:", secondary_pressure)

Once the values are projected to the secondary grid we are able to compare whether we
obtain the expected values or not. Specifically, we expect that the values on the
secondary grid is equal to the sum of the values on each of the mortar sides:

In [34]:
expected_pressure_on_secondary_grid = (mortar_pressure[:4] + mortar_pressure[4:])
print("Expected values on the secondary grid:", expected_pressure_on_secondary_grid)

Expected values on the secondary grid: [1.37739803 1.33558773 1.62133727 0.65985094]


Finally, we check that the values match:

In [35]:
assert np.isclose(
    secondary_pressure, 
    expected_pressure_on_secondary_grid
).all(), "Arrays are not close enough"